## A Million News Headline dataset

In this notebook, we run NMF, NCPD, LDA, and online NCPD as dynamic topic modeling techniques for the Australian Broadcasting Corporation (ABC) headlines dataset ('A Million News Headline' dataset downloaded from [Kaggle](https://www.kaggle.com/therohk/million-headlines)).

The dataset consists of a million news headlines published over a period of seventeen years in ABC and covers international news along with Australian news from early-2003 to end-2019.

### Standard imports

In [ ]:
import os
import numpy as np
import pickle
import pandas as pd
import tensorly as tl
import nltk
import collections
import gensim
from gensim import corpora,models
from nltk.corpus import stopwords
from tensorly import fold
from tensorly.decomposition import non_negative_parafac
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from config import data_dir, results_dir
from covid19 import plotting, utils
from covid19.online_CPDL.ocpdl import Online_CPDL

In [ ]:
rank = 25
n_top_words = 5 # number of keywords to display for each topic
save_figures = True # True to save results; False otherwise
load_NMF_factors = False # True for loading pre-saved NMF factors; False for computing the factors
load_NCPD_factors = False # True for loading pre-saved NCPD factors; False for computing the factors
load_lda_model = False # True for loading trained LDA model; False for training a model
load_ONCPD_factors = False # True for loading pre-saved ONCPD factors; False for computing the factors
local_path = results_dir
directory = os.path.join(
            data_dir,
            "abcnews-date-text.csv",
        ) # Fill
np.random.seed(0)

### Load ABC news dataset

In [ ]:
data = pd.read_csv(directory, error_bad_lines=False, warn_bad_lines = True);
data.head()

In [ ]:
data.dtypes

In [ ]:
# Format the date of publishing
data['publish_date'] = pd.to_datetime(data['publish_date'],format='%Y%m%d')
data.head()

In [ ]:
print(f"Number of headlines {len(data)}.")

### Randomly subsample dataset

In [ ]:
# Subsample dataset per month
data_subsample = data.copy()
sample_val = 700 # number of headlines per month

sub_indices = data['publish_date'].groupby([data.publish_date.dt.year, data.publish_date.dt.month]).sample(n=sample_val, replace = False).index

data_subsample = data_subsample.iloc[sub_indices]
print(f"Number of headlines in the downsampled dataset {len(data_subsample)}.")

In [ ]:
# Sort data by chronological order 
data_subsample.sort_values(by=['publish_date'], inplace=True)
data_subsample.head()

In [ ]:
# Check min/max number of headlines sampled per month
dfg = data_subsample.groupby([data.publish_date.dt.year, data.publish_date.dt.month])
print(dfg.count().min())
print(dfg.count().max())

In [ ]:
# Create a yyyy-mm column
data_subsample['month_year'] = pd.to_datetime(data_subsample['publish_date']).dt.to_period('M')
data_subsample.head()

In [ ]:
# Test example for monthly subsampling
len(data_subsample[data_subsample["month_year"] == pd.to_datetime('200709', format='%Y%m').to_period('M')])

In [ ]:
# Convert the headline text in the dataframe into a corpus (list)
corpus = data_subsample['headline_text'].values.tolist()
corpus

### Extract TF-IDF weights and features

In [ ]:
# Vary the choices of the parameters of TfidfVectorizer
n_features = 7000
max_df=0.7
min_df=5
stop_words_list = nltk.corpus.stopwords.words('english')
stop_words_list.append("abc") # remove the common string "abc" which could stand for "Australian Broadcasting Corporation"
vectorizer = TfidfVectorizer(max_df=max_df, 
                                min_df=min_df, 
                                #token_pattern = '[a-zA-Z]+',
                                max_features=n_features,
                                #ngram_range = (1,2),
                                stop_words=stop_words_list)
vectors = vectorizer.fit_transform(corpus) 
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = np.array(dense).transpose() 

In [ ]:
print("Shape of the matrix {}.".format(denselist.shape))
print("Dimensions: (vocabulary, headlines)")

In [ ]:
# Organize data into a third-order tensor
num_time_slices = dfg.ngroups # number of time slices
n_div = 1 # number of months per time slice
n_headlines = sample_val*n_div # number of headlines per time slice
data_tens = fold(denselist, 1, (denselist.shape[1] // n_headlines, denselist.shape[0], n_headlines))

print("Shape of the tensor {}.".format(data_tens.shape))
print("Dimensions: (time, vocabulary, headlines)")

In [ ]:
# List of all features/words extracted
feature_names

### Plotting Code

In [ ]:
sns.set(style="whitegrid", font_scale=1.001, context="talk")

def heatmap(
    data,
    x_tick_labels=None,
    x_label="",
    y_tick_labels=None,
    y_label="",
    figsize=(7, 9),
    max_data=None,
):
    """Plot heatmap.
    Args:
        data: (2d array) data to be plotted (topics x date)
        x_tick_labels (list of str)
    Returns:
        fig
        ax
    """

    fig = plt.figure(figsize=figsize)
    ax = sns.heatmap(
        data,
        rasterized=True,
        vmax=max_data,
        cbar_kws=dict(use_gridspec=False, location="top", aspect =80),
        )

    plt.xticks(rotation=45)
    plt.yticks(np.arange(0, data.shape[0], 1.0) + 0.5, rotation=0)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    
    ax.set_xticks(np.arange(0, data.shape[1], 12))
    ax.set_xticklabels([str(x) for x in range(2003, 2020)])

    if y_tick_labels is None:
        y_tick_labels = [topic_num + 1 for topic_num in range(data.shape[0])]
    ax.set_yticklabels(y_tick_labels)

    #if x_tick_labels is not None:
     #   labels = [x_tick_labels[int(item.get_text())] for item in ax.get_xticklabels()]
     #   ax.set_xticklabels(labels)

    return fig, ax

### Run NMF

In [ ]:
# Run NMF
if load_NMF_factors:
    W, H = pickle.load(open(os.path.join(local_path,"NMF_factors_headlines.pickle"), "rb"))
else:
    nmf = NMF(n_components=rank, init='nndsvd')
    W = nmf.fit_transform(denselist) # Dictionary
    H = nmf.components_ # Topic representations
    NMF_factors = W,H
    with open(os.path.join(local_path,"NMF_factors_headlines.pickle"), "wb") as f:
        pickle.dump(NMF_factors, f)

In [ ]:
# Display topics
for topic_idx, topic in enumerate(W.T):
    message = "Topic %d: " % (topic_idx+1)
    message += ", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)

In [ ]:
# Topic Keywords representations
num_keywords = 3
topics_freqs = []
for i, topic in enumerate(W.T):
    topics_freqs.append(
        {feature_names[i]: topic[i] for i in reversed(topic.argsort()[-20:])}
    )

# Make word and frequency lists for each topic.
sorted_topics = [
    sorted(topic.items(), key=lambda item: item[1], reverse=True)
    for topic in topics_freqs
]
word_lists = [[item[0] for item in topic[:num_keywords]] for topic in sorted_topics]
freq_lists = [[item[1] for item in topic[:num_keywords]] for topic in sorted_topics]

y_tick_labels = [
    "{}: {}".format(", ".join(word_lists[i][0:3]), i + 1) for i in range(rank)
]

In [ ]:
# Get topic distributions for each time slice
topics_over_time = np.split(H, num_time_slices, axis=1)

# Average topic distributions over time.
avg_topics_over_time = [np.mean(topics, axis=1) for topics in topics_over_time]

# Normalize to get the distribution over topics for each day.
avg_topics_over_time = np.array(
    [topics / np.sum(topics) for topics in avg_topics_over_time]
)

In [ ]:
#Visualize topic distributions
date_strs = [str(x+1) for x in range(data_tens.shape[0])] 
#y_tick_labels = ["{}".format(i + 1) for i in range(rank)]

fig, ax = heatmap(
    avg_topics_over_time.T,
    x_tick_labels=date_strs,
    x_label="Year",
    y_tick_labels=y_tick_labels,
    y_label="Topic",
    figsize= (10, 8)
)

if save_figures:
    plt.savefig(os.path.join(local_path,"NMF_Normalized_Topic_Time_Headlines.png"), bbox_inches = 'tight', pad_inches = 0)
plt.show()

### Run NCPD on Tensor data

In [ ]:
if load_NCPD_factors:
    factors = pickle.load(open(os.path.join(local_path,"NCPD_factors_headlines.pickle"), "rb"))
    weights = pickle.load(open(os.path.join(local_path,"NCPD_weights_headlines.pickle"), "rb"))
else:
    factors = non_negative_parafac(data_tens, rank=rank, init = 'svd') 
    if len(factors) == 2:
        weights, factors = factors
        print(weights)
        with open(os.path.join(local_path,"NCPD_weights_headlines.pickle"), "wb") as f:
            pickle.dump(weights, f)
    with open(os.path.join(local_path,"NCPD_factors_headlines.pickle"), "wb") as f:
        pickle.dump(factors, f)

In [ ]:
# Dimensions of the factor matrices
for i in range(len(factors)):
    print(factors[i].shape)

In [ ]:
# Display Topics
for topic_idx, topic in enumerate(factors[1].T):
    message = "Topic %d: " % (topic_idx+1)
    message += ", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)

In [ ]:
# Topic Keywords representations
num_keywords = 3
topics_freqs = []
for i, topic in enumerate(factors[1].T):
    topics_freqs.append(
        {feature_names[i]: topic[i] for i in reversed(topic.argsort()[-20:])}
    )

# Make word and frequency lists for each topic.
sorted_topics = [
    sorted(topic.items(), key=lambda item: item[1], reverse=True)
    for topic in topics_freqs
]
word_lists = [[item[0] for item in topic[:num_keywords]] for topic in sorted_topics]
freq_lists = [[item[1] for item in topic[:num_keywords]] for topic in sorted_topics]

y_tick_labels = [
    "{}: {}".format(", ".join(word_lists[i][0:3]), i + 1) for i in range(rank)
]

In [ ]:
# Plotting normalized temporal factor
time_norm = factors[0]/factors[0].sum(axis=1)[:,None] # row sum equal to 1

fig, ax = heatmap(
    time_norm.T, #factors[0].T for un-normalized factor
    x_tick_labels=date_strs,
    x_label="Year",
    y_tick_labels=y_tick_labels,
    y_label="Topic",
    figsize=(10, 8)
)

if save_figures:
    plt.savefig(os.path.join(local_path,"NCPD_Normalized_Topic_Time_Headlines.png"), bbox_inches = 'tight', pad_inches = 0)
plt.show()

## LDA + Temporal evolution extraction

In [ ]:
# Data Preprocessing
tokenized_docs = list(map(vectorizer.build_analyzer(),corpus)) # pre-process and tokenize documents using tfidfvectorizer
# Filter processed_docs based on features list
processed_docs = []
for doc in tokenized_docs:
    processed_docs.append([wd_str for wd_str in doc if(wd_str in feature_names)])
                              
dictionary = gensim.corpora.Dictionary(processed_docs) # define a (gensim) dictionary
print(len(dictionary))

# Create bag-of-words corpus (gensim format)
corpus=[dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
# Alternative Preprocessing (to create a separate dictionary from NMF/NCPD)
"""
# Data Preprocessing
processed_docs = list(map(vectorizer.build_analyzer(),corpus)) # pre-process and tokenize documents using tfidfvectorizer
dictionary = gensim.corpora.Dictionary(processed_docs) # define a (gensim) dictionary
dictionary.filter_extremes(no_below=min_df,no_above=max_df,keep_n=n_features) # use the same parameters as tfidfvectorizer
print(len(dictionary))

# Create bag-of-words corpus (gensim format)
corpus=[dictionary.doc2bow(doc) for doc in processed_docs]
"""

In [ ]:
# Check if dictionaries are the same
collections.Counter(list(dictionary.values())) == collections.Counter(feature_names)

In [ ]:
# Run LDA
run_LDA_tfidf = False

if run_LDA_tfidf:
    # Convert BOW Corpus to TFIDF Corpus
    tfidf=models.TfidfModel(corpus)
    corpus=tfidf[corpus]   

if load_lda_model:
    lda_model = pickle.load(open(os.path.join(local_path,"lda_model_headlines.pickle"), "rb"))
else:
    lda_model = gensim.models.LdaMulticore(corpus,
                                         num_topics=rank, 
                                         id2word = dictionary, 
                                         passes = 20,
                                         workers=4,
                                         minimum_probability = 0,
                                         random_state = 1
                                          )
    
    with open(os.path.join(local_path,"lda_model_headlines.pickle"), "wb") as f:
        pickle.dump(lda_model, f)                 

In [ ]:
# Display topics
for topic_idx, topic in enumerate(lda_model.get_topics()):
    message = "Topic %d: " % (topic_idx+1)
    message += ", ".join([dictionary[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)

In [ ]:
# Topic Keywords representations
num_keywords = 3
topics_freqs = []
for i, topic in enumerate(lda_model.get_topics()):
    topics_freqs.append(
        {dictionary[i]: topic[i] for i in reversed(topic.argsort()[-20:])}
    )

# Make word and frequency lists for each topic.
sorted_topics = [
    sorted(topic.items(), key=lambda item: item[1], reverse=True)
    for topic in topics_freqs
]
word_lists = [[item[0] for item in topic[:num_keywords]] for topic in sorted_topics]
freq_lists = [[item[1] for item in topic[:num_keywords]] for topic in sorted_topics]

y_tick_labels = [
    "{}: {}".format(", ".join(word_lists[i][0:3]), i + 1) for i in range(rank)
]

In [ ]:
# LDA representation matrix (topics by documents)
lda_rep_mat = np.empty((rank, len(corpus)))
for numb in range(len(corpus)):
    for index, score in lda_model.get_document_topics(corpus[numb], minimum_probability = 0):
        lda_rep_mat[index, numb] = score

In [ ]:
# Get topic distributions for each time slice
lda_topics_over_time = np.split(lda_rep_mat, num_time_slices, axis=1)

# Average topic distributions over time.
lda_avg_topics_over_time = [np.mean(topics, axis=1) for topics in lda_topics_over_time]

# Normalize to get the distribution over topics for each day.
lda_avg_topics_over_time = np.array(
    [topics / np.sum(topics) for topics in lda_avg_topics_over_time]
)

In [ ]:
#Visualize topic distributions
fig, ax = heatmap(
    lda_avg_topics_over_time.T,
    x_tick_labels=date_strs,
    x_label="Year",
    y_tick_labels=y_tick_labels,
    y_label="Topic",
    figsize=(10, 8)
)

if save_figures:
    plt.savefig(os.path.join(local_path,"LDA_Topic_Time_Headlines.png"), bbox_inches = 'tight', pad_inches = 0)
plt.show()

## Online NCPD

In [ ]:
# SVD intialization (Optional)
weights, factors = non_negative_parafac(data_tens, rank=rank, init = 'svd', n_iter_max=1) 
print(factors[0].shape)
print(factors[1].shape)
loading = {}
loading.update({'U0':factors[0]})
loading.update({'U1':factors[1]})

In [ ]:
# Run Online NCPD
if load_ONCPD_factors:
    factors = pickle.load(open(os.path.join(local_path,"20news_ONCPD_factors.pickle"), "rb"))
else:
    OCPDL_model = Online_CPDL(X=data_tens,
                        batch_size=data_tens.shape[-1]//10, # use only 1/10 documents per minibatch
                        iterations=20,
                        n_components=rank,
                        ini_loading=loading,
                        ini_A=None,
                        ini_B=None,
                        alpha=0,
                        beta=1,
                        subsample=True)
    
    result_dict = OCPDL_model.train_dict(if_compute_recons_error=False,
                                   save_folder=None,
                                   output_results=False)
    loading = result_dict.get("loading")
    factors = []
    for i in loading.keys():
        factors.append(loading.get(str(i)))
    
    print('!!! X.shape', data_tens.shape)
    with open(os.path.join(local_path,"20news_ONCPD_factors.pickle"), "wb") as f:
        pickle.dump(factors, f)
    

In [ ]:
# Obtain the shape of the factor matrices
for i in range(len(factors)):
    print(factors[i].shape)

In [ ]:
# Display topics
for topic_idx, topic in enumerate(factors[1].T):
    message = "Topic %d: " % (topic_idx+1)
    message += ", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)

In [ ]:
#Visualize topic distributions
date_strs = [str(x+1) for x in range(data_tens.shape[0])]

time_norm = factors[0]/factors[0].sum(axis=1)[:,None] # row sum equal to 1
y_tick_labels = ["{}".format(i + 1) for i in range(rank)]

fig, ax = heatmap(
    time_norm.T, #factors[2].T for un-normalized factor
    x_tick_labels=date_strs,
    x_label="Time",
    y_tick_labels=y_tick_labels,
    y_label="",
    figsize = (6,4),
)


if save_figures:
    plt.savefig(os.path.join(local_path,"ONCPD_Normalized_Topic_Time_Headlines.png"), bbox_inches = 'tight', pad_inches = 0)
plt.show()